In [1]:
# !pip install -U git+https://github.com/openfisca/openfisca-ceq.git@master#egg=OpenFisca-CEQ
# !pip install -U git+https://github.com/openfisca/openfisca-senegal.git@ceq#egg=OpenFisca-Senegal
# !pip install -U git+https://github.com/openfisca/openfisca-mali.git@master#egg=OpenFisca-Mali
# !pip install -U git+https://github.com/openfisca/openfisca-cote-d-ivoire.git@master#egg=OpenFisca-COTE-D-IVOIRE

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data.tax_targets import read_tax_target
from openfisca_ceq.tools.results.inequality import inequality_table, incidence_table, concentration_share, taxpayers_share
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )
from openfisca_ceq.tools.tax_benefit_system_ceq_completion import labor_type_by_index

In [3]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable['autres_revenus_du_capital_brut'] = 'autres_revenus_du_capital_brut'
ceq_by_harmonized_variable['pension_retraite_brut'] = 'pension_retraite_brut'
ceq_by_harmonized_variable['revenu_foncier_brut'] = 'revenu_foncier_brut'
ceq_by_harmonized_variable['revenu_non_salarie_brut'] = 'revenu_non_salarie_brut' 
ceq_by_harmonized_variable['salaire_brut'] = 'salaire_brut'
ceq_by_harmonized_variable['salaire_super_brut'] = 'salaire_super_brut'


In [4]:
def income_table(survey_scenario, inflated_survey_scenario):
    data = [
        (
            harmonized_variable, 
            openfisca_variable, 
            survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            inflated_survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            ) 
        for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
        ]
    revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate", "inflated aggregate"])
    
    return revenus.round(0).astype({"aggregate": int, "inflated aggregate": int})

In [5]:
def wage_earner(survey_scenario):
    df = pd.concat(
        [
            (
                survey_scenario.compute_pivot_table(
                    aggfunc = 'sum', 
                    values = ["salaire_brut", "salaire_super_brut"], 
                    index = "secteur_public", 
                    period = survey_scenario.year, 
                    concat_axis = 1
                    ) / 1e9
                ).round(0).astype(int)
            ],
        axis = 1,
        )
    df.loc[True, "effectifs"] = (
        (survey_scenario.calculate_variable('salaire_brut', period =survey_scenario.year) > 0) 
        * survey_scenario.calculate_variable('secteur_public', period =survey_scenario.year)
        * survey_scenario.calculate_variable('person_weight', period =survey_scenario.year)
        ).sum()
    df.loc[False, "effectifs"] = (
        (survey_scenario.calculate_variable('salaire_brut', period =survey_scenario.year) > 0) 
        * np.logical_not(survey_scenario.calculate_variable('secteur_public', period =survey_scenario.year))
        * survey_scenario.calculate_variable('person_weight', period = survey_scenario.year)
        ).sum()
    return df.round(0).astype(int)

In [6]:
def income_distribution(survey_scenario):
    survey_income = (
        survey_scenario.compute_pivot_table(
            values = "survey_income", columns = "decile_gross_income_per_capita", period = survey_scenario.year
            ).transpose().round()    
        )
    gross_income = (
        survey_scenario.compute_pivot_table(
            values = "gross_income", columns = "decile_gross_income_per_capita", period = survey_scenario.year
            ).transpose().round()
        )
    return pd.concat([survey_income, gross_income], axis = 1)

## Inequality tables

In [ ]:
for country, year in year_by_country.items():
    survey_scenario = build_ceq_survey_scenario(country, year)
    inflated_survey_scenario = build_ceq_survey_scenario(country, year, inflate = True)
    display(country)
    display(income_table(survey_scenario, inflated_survey_scenario))
    display(wage_earner(survey_scenario))

/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [ ]:
income_variable = 'gross_income'
by_variable = 'decile_{}_per_capita'.format(income_variable)
tax_variables = ['personal_income_tax', 'customs_duties', 'value_added_tax', 'education_net_transfers']

table_by_indicator_by_scenario = dict()

for survey_type in ["bare" , "inflated"]:
    survey_scenario = build_ceq_survey_scenario(country, year, inflate = survey_type == "inflated")
    income_by_country = dict()
    gini_by_country = dict()
    incidence_by_country = dict()
    concentration_share_by_country = dict()
    taxpayers_share_by_country = dict()
    for country, year in year_by_country.items():
        survey_scenario = build_ceq_survey_scenario(country, year)
        income_by_country[country] = income_distribution(survey_scenario)
        gini_by_country[country] = inequality_table(survey_scenario)
        incidence_by_country[country] = incidence_table(survey_scenario, income_variable, tax_variables, by_variable)
        concentration_share_by_country[country] = concentration_share(survey_scenario, tax_variables, by_variable)  
        taxpayers_share_by_country[country] = taxpayers_share(survey_scenario, tax_variables, by_variable)
        display('incidence - ' + country + ' - ' + survey_type)
        display(incidence_table(survey_scenario, income_variable, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        display('concentration_share - ' + country + ' - ' + survey_type)
        display(concentration_share(survey_scenario, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        display('taxpayers_share - ' + country + ' - ' + survey_type)
        display(taxpayers_share(survey_scenario, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        table_by_indicator = dict(
            ginis = pd.concat(gini_by_country).swaplevel().sort_index(),
            incidences = pd.concat(incidence_by_country).transpose().stack(),
            concentrations = pd.concat(concentration_share_by_country).transpose().stack(),
            taxpayers = pd.concat(taxpayers_share_by_country).transpose().stack(),
            )
        table_by_indicator_by_scenario[survey_type] = table_by_indicator

In [ ]:
sheet_name_by_indicator = dict(
    ginis = "r1",
    incidences = "r2",
    concentrations = "r3",
    taxpayers = "r4",
    )
for survey_type, table_by_indicator in table_by_indicator_by_scenario.items():
    with pd.ExcelWriter(
        '/home/benjello/Dropbox/Projet_Micro_Sim/resultats/graphs-tableaux/Resultats_Taxineq_{}.xlsx'.format(survey_type)
        ) as writer:
            for indicator, table in table_by_indicator.items(): 
                display(indicator + " - " + survey_type)
                display(table)
                table.to_excel(writer, sheet_name = sheet_name_by_indicator[indicator])

In [ ]:
result = read_tax_target(add_country_details=True)
display(result)